# Preleminaries

- write intro or I don't know
- cell below used for preleminaries, resp. import of possible libraries used in this project
- maybe note what our "analysis question is"

In [1]:
import os
import pandas as pd # for data stuff
import numpy as np # for importing data i think, not sure anymore
import matplotlib.pyplot as plt #for plotting

# the modules below are used in "Analys Models" section
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix


Install Brawl Stars API with pip
 - https://brawlstats.readthedocs.io/en/latest/index.html

In [2]:
!pip install brawlstats

# Data Collection

- collect Data from brawl stars API

Import brawlstats 

In [3]:
import brawlstats
import time

Establish connection with token to Brawl Stars API
- create account at: https://developer.brawlstars.com/#/
- get ip with: https://www.whatismyip.org/
- create API Key on https://developer.brawlstars.com/#/ for your account and your API Key
- save API key locally in a file at: ../token_brawlstats.txt (so it is not part of this git repository)

In [4]:
path = os.getcwd()
os.chdir("..")
path_one_up = os.getcwd()
os.chdir(path)

token_file =  open(os.path.join(path_one_up, "token_brawlstats.txt"), 'r')
token = token_file.read()

client = brawlstats.Client(token)

In [5]:
tags_top_200_players = [player["tag"] for player in client.get_rankings(ranking='players', limit=200)]
top_200_players = [client.get_player(tag) for tag in tags_top_200_players]

    

↻さー͛が↺￤Puu!! (#9U2QJ0P)


In [10]:
print([{brawler["name"]:brawler["trophies"]} for brawler in top_200_players[0].brawlers])

[{'SHELLY': 1001}, {'COLT': 1102}, {'BULL': 1005}, {'BROCK': 1101}, {'RICO': 1000}, {'SPIKE': 1002}, {'BARLEY': 1005}, {'JESSIE': 952}, {'NITA': 1004}, {'DYNAMIKE': 1004}, {'EL PRIMO': 1000}, {'MORTIS': 1020}, {'CROW': 924}, {'POCO': 1004}, {'BO': 967}, {'PIPER': 1040}, {'PAM': 1003}, {'TARA': 1002}, {'DARRYL': 1005}, {'PENNY': 1100}, {'FRANK': 912}, {'GENE': 1104}, {'TICK': 910}, {'LEON': 1002}, {'ROSA': 1003}, {'CARL': 905}, {'BIBI': 1101}, {'8-BIT': 1005}, {'SANDY': 1000}, {'BEA': 916}, {'EMZ': 1141}, {'MR. P': 905}, {'MAX': 1005}, {'JACKY': 1005}, {'GALE': 930}, {'SPROUT': 1160}]


# Data Processing
- maybe just plot the data, see if there is somethin weird

In [ ]:
print("whatever this is")

# Data Cleaning

- maybe write down what we figured out and how we got rid of it

In [ ]:
print("Is e dreckig, musse putze")

# Preparation for Model Building

- do some stratification perhaps

In [ ]:
print("stratify data for training purposes")

# Train Model

In [ ]:
print("run code which uses train data to train the model")

# Test Model

In [ ]:
print("run code to test the model")

# Analyse Model

- probably some plots will be excpected

In [ ]:



# import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target
class_names = iris.target_names

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Run classifier, using a model that is too regularized (C too low) to see
# the impact on the results
classifier = svm.SVC(kernel='linear', C=0.01).fit(X_train, y_train)

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(classifier, X_test, y_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

# Visualize results

In [ ]:
print("some very nice plots like the ones above")

# Discussion or whatever

- write down our insights or maybe do some further coding in the cell below

In [ ]:
print("finally, done with this")